In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from template import pca_function
import matplotlib.pyplot as plt
import statsmodels.api as sma
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
#importing the necessary libraries to use for analysis
# Question 1
# reading the first dataset 
df=pd.read_csv("2021-12.csv")
# dropping the row that contain NaN in all columns
df=df.dropna(how='all')
df = df.drop(labels=1, axis=0)
#resetting the index after dropping
df=df.reset_index()
#dropping the index column
df=df.drop(['index'], axis=1)
#converting the 'sasdate' column from string type to date time format
df['sasdate'] = pd.to_datetime(df['sasdate'])
df.index = df['sasdate']
#dropping the 'sasdate' column from the dataset
df = df.drop(['sasdate'], axis=1)
#Question 2
# dropping the data after december 2019 by first getting row index and then droping row after that index
df = df[df.index <= '2019-12-31']
# Question 3
#reading the second dataset into dataframe 'df2'
df1=pd.read_csv('fred_md_desc.csv')
#dropping the rows that conatin Nan for all columns
df1=df1.dropna(how='all')
# resetting index after dropping 
df1=df1.reset_index().drop(['index'], axis=1)
#extracting the matching values from column and key column named 'fred'
matching_values = [x for x in df1['fred'] if x in df.columns]
df1 = df1[df1['fred'].isin(matching_values)]#selecting only rows that has matching rows
df1 = df1.reset_index().drop(['index'], axis=1)
df2 = df.copy()
#Question 4
# Implementing the transformation table to apply different transformation rules describe in table given in question 4
for i in range(0,len(df1)):
    if df1['tcode'][i]==1:
        s=df1['fred'][i]
        srs=df[s]    
        df2.loc[:,s]=srs
    elif df1['tcode'][i]==2:
        s=df1['fred'][i]
        srs=df[s]    
        df2.loc[:,s]=srs.diff()
    elif df1['tcode'][i]==3:
        s=df1['fred'][i]
        srs=df[s]    
        df2.loc[:,s]=srs.diff().diff()
    elif df1['tcode'][i]==4:
        s=df1['fred'][i]
        srs=df[s]    
        df2.loc[:,s]=np.log(srs)
    elif df1['tcode'][i]==5:
        s=df1['fred'][i]
        srs=df[s]    
        df2.loc[:,s]=np.log(srs).diff()
    elif df1['tcode'][i]==6:
        s=df1['fred'][i]
        srs=df[s]    
        df2.loc[:,s]=np.log(srs).diff().diff()
    elif df1['tcode'][i]==7:
        s=df1['fred'][i]
        srs=df[s]    
        df2.loc[:,s]=((srs/srs.shift(1)) - 1)

# Question 6
# Applying the question 6 requiremnet by subtracting mean and then dividing with std and filling na with zeros
df3 = df2-df2.mean()   
df3 = df3/df3.std()
df3 = df3.fillna(0)
#converting the df2 into csv file called 'transformed_data.csv'
df3.to_csv('transformed_data.csv')
# Question 7
#calculting the PCA of transformed dataset
factor = pca_function(df3)
# Question 8
figure, axis = plt.subplots(2, 1)
  
axis[0].hist(factor['comp_0'])
axis[0].set_title("Distribution of Factor")
  
axis[1].plot(factor['comp_0'])
axis[1].set_title("Time Series")

figure.tight_layout()

figure.savefig("factor.pdf", bbox_inches='tight')
#Question 9
# shift the untransformed data by value of 1
df4 = df2.shift(1)
x = factor.shift(1).fillna(0)
# Question 10
#traing the model with 5 different variables and storing their fitted values 
y = df4['INDPRO'].fillna(0)
model1 = sma.OLS(y,x).fit()
indpro=model1.fittedvalues)
y = df4['S&P 500'].fillna(0)
model2 = sma.OLS(y,x).fit()
SP=model2.fittedvalues
y = df4['PAYEMS'].fillna(0)
model3 = sma.OLS(y,x).fit()
paymes=model3.fittedvalues
y = df4['CPIAUCSL'].fillna(0)
model4 = sma.OLS(y,x).fit()
cpiaucsl=model4.fittedvalues
y = df4['BUSINVx'].fillna(0)
model5 = sma.OLS(y,x).fit()
businvx=model5.fittedvalues
data={'INDPRO_fitted':indpro,'S&P 500_fitted':SP,'PAYEMS_fitted':paymes,'CPIAUCSL_fitted':cpiaucsl,'BUSINVx_fitted':businvx}
df5 = pd.DataFrame(data) 

# Question 11
#saving the fitted values into a csv file named 'fitted_values.csv'
df5.to_csv('fitted_values.csv')

# Question 12
#reading the third dataset 
df_nber = pd.read_csv('NBER_DATES.csv')
#transforming the dataset
df_transformed_fitted = pd.concat([df3[['INDPRO','S&P 500', 'PAYEMS', 'CPIAUCSL', 'BUSINVx']], df5], axis=1)
df_transformed_fitted = df_transformed_fitted.merge(df_nber['0'], left_index=True, right_index=True)
plot = sns.lmplot(x='INDPRO', y='INDPRO_fitted',col='0', data=df_transformed_fitted)
#INDPRO
plot.fig.suptitle(df1[df1['fred'] == 'INDPRO']['description'].to_list()[0], fontsize=15)
fig = plot.figure
fig.tight_layout()
#saving the file in pdf for INDPRO
fig.savefig('INDPRO.pdf')
plot = sns.lmplot(x='S&P 500', y='S&P 500_fitted',col='0', data=df_transformed_fitted)
plot.fig.suptitle(df1[df1['fred'] == 'S&P 500']['description'].to_list()[0], fontsize=15)
fig = plot.figure
fig.tight_layout()
fig.savefig('S&P 500.pdf')
plot = sns.lmplot(x='PAYEMS', y='PAYEMS_fitted',col='0', data=df_transformed_fitted)
plot.fig.suptitle(df1[df1['fred'] == 'PAYEMS']['description'].to_list()[0], fontsize=15)
fig = plot.figure
fig.tight_layout()
fig.savefig('PAYEMS.pdf')
plot = sns.lmplot(x='CPIAUCSL', y='CPIAUCSL_fitted',col='0', data=df_transformed_fitted)
plot.fig.suptitle(df1[df1['fred'] == 'CPIAUCSL']['description'].to_list()[0], fontsize=15)
fig = plot.figure
fig.tight_layout()
fig.savefig('CPIAUCSL.pdf')
plot = sns.lmplot(x='BUSINVx', y='BUSINVx_fitted',col='0', data=df_transformed_fitted)
plot.fig.suptitle(df1[df1['fred'] == 'BUSINVx']['description'].to_list()[0], fontsize=15)
fig = plot.figure
fig.tight_layout()
fig.savefig('BUSINVx.pdf')
